## Introduction

This *optimization* file will contain different optimization algorithm, which objective will be to act on the battery in order to reduce the daily electricity bill. 

We will first implement a **genetic algorithm**, which proceeds blindly, testing daily battery’s charging and discharging profiles and selecting the ones that lead to better results in terms of savings. Throughout the generations, the best profiles become the “parents”, they originate children by crossing their DNA, and the best children then become the parents of the next generation.


In [70]:
from ipynb.fs.full.Controller import controller
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt

In [100]:
class Agent:
    #E_L: daily load, ie client energy consumption
    #E_S: daily solar output
    #SOC_init: inital state of charge of the battery, at t=0
    #k_buy & k_sell: daily electricty buy and sell prices
    
    def __init__(self, E_L, E_S, SOC_init, k_buy, k_sell):
        #initialize daily grid power output instructions
        #only positive for now as I didn't implement the buy from grid equations in the controller
        self.P_G_obj = np.random.randint(0, 50, len(E_L))
        print('initial P_G_obj = ', self.P_G_obj)
        self.fitness = 0
        #initialize daily battery's SOC profile
        SOC_ = np.concatenate((SOC_init, np.zeros(len(E_L)-1)), axis=None)
        
        #calculte the first term, and the initial SOC for the next iteration
        self.P_G_obj[0], SOC_[1], cost_0 = controller(E_L[0], E_S[0], self.P_G_obj[0], SOC_[0],\
                                              k_buy, k_sell)
        self.fitness = cost_0
        
        #calculate all other terms, using previously computed SOC as an input
        for i in range(1, len(self.P_G_obj)):
            self.P_G_obj[i], SOC_[i+1], cost_i  = controller(E_L[i], E_S[i], self.P_G_obj[i], SOC[i], k_buy, k_sell)
            #apply_controller = [controller(E_L[i], E_S[i], self.P_G_obj[i], SOC[i], k_buy, k_sell) for i in range(len(E_L))]    
            
            #fitness has to be low for the profile to be competitive (= low energy cost)
            self.fitness += cost_i
            
        print('P_G_obj after controller = ', self.P_G_obj)
        
        #idky but fitness is a vector, hence we have to sum all its components
        self.fitness = sum(self.fitness)

        print('P_G_obj =',  self.P_G_obj )
        print('fitness =', self.fitness)
        
        #plt.figure()
        #plt.plot(range(len(E_L)), self.P_G_obj)
        #plt.plot(range(len(E_L)), k_buy)
        #plt.title("Oui")
        #plt.legend(['P_G_obj', 'electricity cost'])
    
    
    def _str_(self):
        
        return 'Grid power instruction: ' + self.P_G_obj + ' Fitness :' + str(self.fitness)
    

population = 1
generations = 1

def ga():
    
    agents = init_agents(population, E_L, E_S, SOC, k_buy, k_sell)
    
    for generation in range(generations):
        print('\n')
        print('Generation' + str(generation))
        
        agents = selection(agents)
        agents = crossover(agents)
        agents = mutation(agents)
        
    return sorted(agents, key=lambda agent: agent.fitness, reverse = False)[0].P_G_obj
        
        
def init_agents(population, E_L, E_S, SOC, k_buy, k_sell):
    
    return [Agent(E_L, E_S, SOC,k_buy, k_sell) for _ in range(population)]


def selection(agents):
    print('Agent 0 before sorting', agents[0].P_G_obj, '\n')
    agents = sorted(agents, key=lambda agent: agent.fitness, reverse = False)
    print('Agent 0 after sorting', agents[0].P_G_obj, '\n')
    print(len(agents))
    agents = agents[:int(0.5 * len(agents))]
    print(len(agents))
    
    return agents


def crossover(agents):
        
        offspring = []
        
        for _ in range(int((population - len(agents)) / 2)):
            
            parent1 = np.random.choice(agents)
            parent2 = np.random.choice(agents)
            child1 = Agent(E_L, E_S, SOC, k_buy, k_sell)
            child2 = Agent(E_L, E_S, SOC, k_buy, k_sell)

            split = np.random.randint(0, len(child1.P_G_obj))

            child1.P_G_obj = np.concatenate((parent1.P_G_obj[:split], parent2.P_G_obj[split:]), axis=None)
            child2.P_G_obj = np.concatenate((parent2.P_G_obj[:split], parent1.P_G_obj[split:]), axis=None)

            offspring.append(child1)
            offspring.append(child2)

        agents.extend(offspring)
        
        return agents
        
def mutation(agents):
        
        for agent in agents:
            
            for idx, param in enumerate(agent.P_G_obj):
                
                if np.random.uniform(0.0, 1.0) <= 0.1:
                    
                    agent.P_G_obj = np.concatenate((agent.P_G_obj[:idx], np.random.randint(-50, 50), \
                                                     agent.P_G_obj[idx+1:]), axis=None)
        
        return agents

In [106]:
population = 10
generations = 100

length = 10
E_L = np.random.randint(0, 50, length)
E_S = np.random.randint(-50, 0, length)
print("E_S = ", E_S)
print("E_L = ", E_L)

print('\n')

SOC_init = 0.1
k_buy = np.random.randint(0, 50, length)
k_sell = k_buy
P_G_obj_test = ga()

E_S =  [-49  -7 -33 -15  -1 -35 -49 -30 -36 -41]
E_L =  [ 6  0 45 27 12 39 15 44 11  6]


initial P_G_obj =  [38 31 18 44 36  8 12 22 36 33]
P_G_obj after controller =  [38 31 18 44 36  8 12 22 36 33]
P_G_obj = [38 31 18 44 36  8 12 22 36 33]
fitness = 4298.033686465378
initial P_G_obj =  [24 41  1 29 19 31 13 13 11  8]
P_G_obj after controller =  [24 41  1 29 19 31 13 13 11  8]
P_G_obj = [24 41  1 29 19 31 13 13 11  8]
fitness = 41647.00137185971
initial P_G_obj =  [13 18 23 42 48 42 35  0  7 32]
P_G_obj after controller =  [13 18 23 42 48 42 35  0  7 32]
P_G_obj = [13 18 23 42 48 42 35  0  7 32]
fitness = 19264.17600285615
initial P_G_obj =  [ 1 32 12 33 27 22 24 11 38 15]
P_G_obj after controller =  [ 1 32 12 33 27 22 24 11 38 15]
P_G_obj = [ 1 32 12 33 27 22 24 11 38 15]
fitness = 29464.028529341052
initial P_G_obj =  [ 8 33 35 16 28 23 31 39  1  6]
P_G_obj after controller =  [ 8 33 35 16 28 23 31 39  1  6]
P_G_obj = [ 8 33 35 16 28 23 31 39  1  6]
fitness = 40980.170845731824
ini

P_G_obj after controller =  [  7   6  10   2  49  20  49  24 -36  45]
P_G_obj = [  7   6  10   2  49  20  49  24 -36  45]
fitness = 12752.379823726593


Generation14
Agent 0 before sorting [-46  47  20 -50  37  -5 -27  28  38 -38] 

Agent 0 after sorting [-46  47  20 -50  37  -5 -27  28  38 -38] 

10
5
initial P_G_obj =  [ 4 18 40  2  1 34 35 11 33 29]
P_G_obj after controller =  [ 4 18 40  2  1 34 35 11 33 29]
P_G_obj = [ 4 18 40  2  1 34 35 11 33 29]
fitness = 21847.73873943518
initial P_G_obj =  [30  6 23 41 48 14 15 32 27 35]
P_G_obj after controller =  [30  6 23 41 48 14 15 32 27 35]
P_G_obj = [30  6 23 41 48 14 15 32 27 35]
fitness = 12477.728907867508
initial P_G_obj =  [10 23 41 37 42 27 19 44 44 33]
P_G_obj after controller =  [ 10  23  41  37  42  27  19  44 -36  33]
P_G_obj = [ 10  23  41  37  42  27  19  44 -36  33]
fitness = 15571.848919072243
initial P_G_obj =  [44  7 31 27 47 18 16 49 11 27]
P_G_obj after controller =  [44  7 31 27 47 18 16 49 11 27]
P_G_obj = [44  7 31 

P_G_obj after controller =  [ 4 20 48 27  6 48 13 32 13 24]
P_G_obj = [ 4 20 48 27  6 48 13 32 13 24]
fitness = 35663.635266634126
initial P_G_obj =  [40 26 17 21 11 44 12 12 11 13]
P_G_obj after controller =  [40 26 17 21 11 44 12 12 11 13]
P_G_obj = [40 26 17 21 11 44 12 12 11 13]
fitness = 31430.66242411547


Generation25
Agent 0 before sorting [ 23 -28  33   1 -49  13 -45 -23  -6  44] 

Agent 0 after sorting [ 23 -28  33   1 -49  13 -45 -23  -6  44] 

9
4
initial P_G_obj =  [29 10 41 34 18 42 14 40 23 30]
P_G_obj after controller =  [29 10 41 34 18 42 14 40 23 30]
P_G_obj = [29 10 41 34 18 42 14 40 23 30]
fitness = 13660.947539577992
initial P_G_obj =  [43 30 43 21 47 32 48 29 19 35]
P_G_obj after controller =  [43 30 43 21 47 32 48 29 19 35]
P_G_obj = [43 30 43 21 47 32 48 29 19 35]
fitness = -16018.206945602124
initial P_G_obj =  [ 1 40 41  7 13 15 47 38 10  8]
P_G_obj after controller =  [ 1 40 41  7 13 15 47 38 10  8]
P_G_obj = [ 1 40 41  7 13 15 47 38 10  8]
fitness = 33313.83

P_G_obj after controller =  [44 22 46  7 14 10 24 36  9 20]
P_G_obj = [44 22 46  7 14 10 24 36  9 20]
fitness = 19631.00652898404
initial P_G_obj =  [46 30  8 34 32  2 15 27 39 24]
P_G_obj after controller =  [46 30  8 34 32  2 15 27 39 24]
P_G_obj = [46 30  8 34 32  2 15 27 39 24]
fitness = 5531.44894952932


Generation34
Agent 0 before sorting [-46  27 -10  40  37 -11   7 -23 -33  44] 

Agent 0 after sorting [-46  27 -10  40  37 -11   7 -23 -33  44] 

10
5
initial P_G_obj =  [43  0 45 18  5  9 13 17 48 17]
P_G_obj after controller =  [ 43   0  45  18   5   9  13  17 -36  17]
P_G_obj = [ 43   0  45  18   5   9  13  17 -36  17]
fitness = 32535.156034891785
initial P_G_obj =  [15 32 36 11 20 28  6 13 14  3]
P_G_obj after controller =  [15 32 36 11 20 28  6 13 14  3]
P_G_obj = [15 32 36 11 20 28  6 13 14  3]
fitness = 50346.70642482953
initial P_G_obj =  [28 28  3 11  4 28 22 12 47  1]
P_G_obj after controller =  [ 28  28   3  11   4  28  22  12 -36   1]
P_G_obj = [ 28  28   3  11   4  2


Generation44
Agent 0 before sorting [-46 -11  28 -11  37  49  22  35  43  44] 

Agent 0 after sorting [-46 -11  28 -11  37  49  22  35  43  44] 

10
5
initial P_G_obj =  [15 44 36 41 49 11 35 43  9 22]
P_G_obj after controller =  [15 44 36 41 49 11 35 43  9 22]
P_G_obj = [15 44 36 41 49 11 35 43  9 22]
fitness = 14814.077687179419
initial P_G_obj =  [14 39 42 25 27 23 19 47 20 13]
P_G_obj after controller =  [14 39 42 25 27 23 19 47 20 13]
P_G_obj = [14 39 42 25 27 23 19 47 20 13]
fitness = 25963.831897987595
initial P_G_obj =  [15 10 47 20  4 45 35  1 34 12]
P_G_obj after controller =  [15 10 47 20  4 45 35  1 34 12]
P_G_obj = [15 10 47 20  4 45 35  1 34 12]
fitness = 22360.947539578006
initial P_G_obj =  [27 34 23 48  7  0 19 10 41 28]
P_G_obj after controller =  [27 34 23 48  7  0 19 10 41 28]
P_G_obj = [27 34 23 48  7  0 19 10 41 28]
fitness = 11981.301476014225


Generation45
Agent 0 before sorting [-46 -11  28 -11  37  49  22  35  43  44] 

Agent 0 after sorting [-46 -11  28 -11

P_G_obj after controller =  [43 35 19 34 37 29 21 29 17 18]
P_G_obj = [43 35 19 34 37 29 21 29 17 18]
fitness = 11364.37263420961
initial P_G_obj =  [40 12 44 46  4 45 16 36  5 28]
P_G_obj after controller =  [40 12 44 46  4 45 16 36  5 28]
P_G_obj = [40 12 44 46  4 45 16 36  5 28]
fitness = 16327.532276514055
initial P_G_obj =  [19 37 49  4 31 38 44 43 47  7]
P_G_obj after controller =  [ 19  37  49   4  31  38  44  43 -36   7]
P_G_obj = [ 19  37  49   4  31  38  44  43 -36   7]
fitness = 13105.313339974553
initial P_G_obj =  [ 0 10  5 32 33 41 37  8 36 25]
P_G_obj after controller =  [ 0 10  5 32 33 41 37  8 36 25]
P_G_obj = [ 0 10  5 32 33 41 37  8 36 25]
fitness = 20994.313644803577
initial P_G_obj =  [ 2 34 11 18 12 22  7 14 13 11]
P_G_obj after controller =  [ 2 34 11 18 12 22  7 14 13 11]
P_G_obj = [ 2 34 11 18 12 22  7 14 13 11]
fitness = 57363.291161765585


Generation56
Agent 0 before sorting [-30 -11  46  27 -22  49  22 -44  43  27] 

Agent 0 after sorting [-30 -11  46  27 -


Generation65
Agent 0 before sorting [ 40 -11  10 -37 -14  49 -16 -44 -38  27] 

Agent 0 after sorting [ 40 -11  10 -37 -14  49 -16 -44 -38  27] 

9
4
initial P_G_obj =  [16 43 30 32 12 10 24  6  4  6]
P_G_obj after controller =  [16 43 30 32 12 10 24  6  4  6]
P_G_obj = [16 43 30 32 12 10 24  6  4  6]
fitness = 44996.90305618299
initial P_G_obj =  [35 22  4  5 24  2 44 37 35  7]
P_G_obj after controller =  [35 22  4  5 24  2 44 37 35  7]
P_G_obj = [35 22  4  5 24  2 44 37 35  7]
fitness = 12981.399791690952
initial P_G_obj =  [25  9 12 22  2 20  2 13 12 20]
P_G_obj after controller =  [25  9 12 22  2 20  2 13 12 20]
P_G_obj = [25  9 12 22  2 20  2 13 12 20]
fitness = 49826.991540292045
initial P_G_obj =  [25  2  6 45  6 44 14 40  2 45]
P_G_obj after controller =  [25  2  6 45  6 44 14 40  2 45]
P_G_obj = [25  2  6 45  6 44 14 40  2 45]
fitness = 28360.652592547805
initial P_G_obj =  [37 16 14 26 45 12 23 23  4  1]
P_G_obj after controller =  [37 16 14 26 45 12 23 23  4  1]
P_G_obj = [

P_G_obj after controller =  [ 27  27  49   0   3  22  25  38 -36  31]
P_G_obj = [ 27  27  49   0   3  22  25  38 -36  31]
fitness = 15622.193023940807


Generation75
Agent 0 before sorting [ 38 -11  10   2  -9  49  19 -44   6  11] 

Agent 0 after sorting [ 38 -11  10   2  -9  49  19 -44   6  11] 

9
4
initial P_G_obj =  [21 48 27 44 36 40 14 48 11 14]
P_G_obj after controller =  [21 48 27 44 36 40 14 48 11 14]
P_G_obj = [21 48 27 44 36 40 14 48 11 14]
fitness = 23163.73358231085
initial P_G_obj =  [18 42 20 34 35 33 36 49 49 11]
P_G_obj after controller =  [ 18  42  20  34  35  33  36  49 -36  11]
P_G_obj = [ 18  42  20  34  35  33  36  49 -36  11]
fitness = 14121.898076910604
initial P_G_obj =  [20  2  5 44 34  3 34 40 37  8]
P_G_obj after controller =  [20  2  5 44 34  3 34 40 37  8]
P_G_obj = [20  2  5 44 34  3 34 40 37  8]
fitness = 23677.53227651406
initial P_G_obj =  [14 24 35 47  7 29 12 48 30 11]
P_G_obj after controller =  [14 24 35 47  7 29 12 48 30 11]
P_G_obj = [14 24 35 47

P_G_obj after controller =  [ 2 25 14 28 43 40 33 26 14 23]
P_G_obj = [ 2 25 14 28 43 40 33 26 14 23]
fitness = 26813.93021366432
initial P_G_obj =  [48 33 38 41 41 10 11  6 13 14]
P_G_obj after controller =  [48 33 38 41 41 10 11  6 13 14]
P_G_obj = [48 33 38 41 41 10 11  6 13 14]
fitness = 20397.492950243362
initial P_G_obj =  [14 17 11 15 40 33 24 14 16  2]
P_G_obj after controller =  [14 17 11 15 40 33 24 14 16  2]
P_G_obj = [14 17 11 15 40 33 24 14 16  2]
fitness = 42247.00137185971
initial P_G_obj =  [44 37 31 13  1 37  8 26 32 43]
P_G_obj after controller =  [44 37 31 13  1 37  8 26 32 43]
P_G_obj = [44 37 31 13  1 37  8 26 32 43]
fitness = 2131.4981073676886


Generation86
Agent 0 before sorting [-46   6 -23  16  48  16 -43  -4   6  41] 

Agent 0 after sorting [-46   6 -23  16  48  16 -43  -4   6  41] 

10
5
initial P_G_obj =  [16  2  9 18 42  7 13  5 19 32]
P_G_obj after controller =  [16  2  9 18 42  7 13  5 19 32]
P_G_obj = [16  2  9 18 42  7 13  5 19 32]
fitness = 40560.554

P_G_obj after controller =  [41 16 12 48 48 15  1 34  5 40]
P_G_obj = [41 16 12 48 48 15  1 34  5 40]
fitness = 19264.17600285615
initial P_G_obj =  [15 48 27 46 49 25 47 19  2 14]
P_G_obj after controller =  [15 48 27 46 49 25 47 19  2 14]
P_G_obj = [15 48 27 46 49 25 47 19  2 14]
fitness = 17914.028529341052
initial P_G_obj =  [18 13 15 16 24 14 14 21 10 41]
P_G_obj after controller =  [18 13 15 16 24 14 14 21 10 41]
P_G_obj = [18 13 15 16 24 14 14 21 10 41]
fitness = 31897.443792404993


Generation95
Agent 0 before sorting [ 12  48  15  16 -40 -23 -13  -4   6  19] 

Agent 0 after sorting [ 12  48  15  16 -40 -23 -13  -4   6  19] 

9
4
initial P_G_obj =  [25 11 39 34 42 43 10 33 25 38]
P_G_obj after controller =  [25 11 39 34 42 43 10 33 25 38]
P_G_obj = [25 11 39 34 42 43 10 33 25 38]
fitness = 9660.996697416362
initial P_G_obj =  [47 34 23 10 28 20 38 13 17 11]
P_G_obj after controller =  [47 34 23 10 28 20 38 13 17 11]
P_G_obj = [47 34 23 10 28 20 38 13 17 11]
fitness = 12147.9353

In [42]:
print(P_G_obj_test)
print(k_buy)

[25 43]
[4 3]


In [ ]:
 
    #on part d'un P_G_obj initial, et d'un SOC initial
    #on déduit le E_G_obj initial
    #on a tous les inputs du controller: E_L, E_S, P_G_obj, E_G_obj, SOC
    
    #Algo gen:
    #on construit un P_G_obj random
    #on fait passer les inputs à la moulinette du controller, avec une boucle sur i (=sur chaque pas de temps d_t)
    #il nous sort les nouveaux E_G et SOC pour chaque pas de temps d_t, ainsi que le vecteur de coûts associés à P_G_obj
    
    #On somme le vec de coût, ça donne un indicateur qu'il faut minimiser. 
    
    #On pourra grapher E_G *3600/d_t et E_L+E_S pour voir l'action de la batterie
    
    